In [1]:
import chromadb
from langchain.document_loaders.pdf import PyPDFDirectoryLoader as PDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings


In [ ]:
Path = "Data"

list_of_chunks = None

def load_pdfs():
    loader = PDFLoader(Path)
    return loader.load()
list_of_docs = load_pdfs()

def split_docs():
    spliter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 80,
        length_function = len,
        is_separator_regex= False)
    return spliter.split_documents(list_of_docs)
list_of_chunks = split_docs()

text_content = "\n\n".join([doc.page_content for doc in list_of_chunks])

# Save to a text file
output_file = "chunk.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(text_content)
def create_id(current_chunk, count):
    return current_chunk.metadata['source'] + "page" + str(current_chunk.metadata['page']) + "chunk" + str(count)

def create_unique_ids(list_of_chunks):
    count = 0
    current_page = 0
    for i in range(len(list_of_chunks)):
        current_chunk = list_of_chunks[i]
        if int(current_chunk.metadata["page"]) > int(current_page):
            count = 0
        current_chunk_id = create_id(current_chunk, count)
        current_chunk.metadata["id"] = current_chunk_id
        count += 1
        current_page = current_chunk.metadata["page"]
    return list_of_chunks

list_of_chunks = create_unique_ids(list_of_chunks)
list_of_ids = [list_of_chunks[i].metadata["id"] for i in range(len(list_of_chunks))]
text_content = "\n\n".join([doc for doc in list_of_ids])

# Save to a text file
output_file = "ids.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(text_content)
print(list_of_ids)
def store_data(list_of_chunks):
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    dataloader = Chroma(
        persist_directory="CHROMA_PATH", embedding_function=embeddings
    )
    dataloader.persist()
    ids = [list_of_chunks[i].metadata["id"] for i in range(len(list_of_chunks))]
    dataloader.add_documents(list_of_chunks, ids = ids)
    return dataloader

dataloader = store_data(list_of_chunks)
print(dataloader)



Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 35 65536 (offset 0)
Ignoring wrong pointing object 1629 65536 (offset 0)
Ignoring wrong pointing object 1639 65536 (offset 0)
Ignoring wrong pointing object 1643 65536 (offset 0)
Ignoring wrong pointing object 1646 65536 (offset 0)
Ignoring wrong pointing object 1649 65536 (offset 0)
Ignoring wrong pointing object 1652 65536 (offset 0)
Ignoring wrong pointing object 1655 65536 (offset 0)
Ignoring wrong pointing object 1659 65536 (offset 0)
Ignoring wrong pointing object 1662 65536 (offset 0)
Ignoring wrong pointing object 1665 65536 (offset 0)
Ignoring wrong pointing object 1668 65536 (offset 0)
Ignoring wrong pointing object 1671 65536 (offset 0)
Ignoring wrong pointing object 1674 65536 (offset 0)
Ignoring wrong pointing object 1677 65536 (offset 0)
Ignoring wrong pointing object 1685 65536 (offset 0)
Ignoring wrong pointing object 1688 65536 (offset 0)
Ignoring wrong pointing object 1691 65536 (offset 0

['Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf00', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf01', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf02', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf10', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf11', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf12', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf20', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf21', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf22', 'Data\\0.2005_-_Eric_J_Miller_-_Atourbasedmodeloftravelmodechoice[retrieved-2015-12-06].pdf30', 'Data\\0.2005_-_Eric_J_Miller_-_Atourba

KeyboardInterrupt: 

In [8]:
print(list_of_ids)

NameError: name 'list_of_ids' is not defined

In [39]:
from langchain.llms.ollama import Ollama
from langchain.prompts import ChatPromptTemplate
format_prompt = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
question = "where can I buy sunburn gel"
results = dataloader.similarity_search_with_score(question, k = 5)
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(format_prompt)
prompt = prompt_template.format(context=context_text, question=question)

model = Ollama(model = "mistral")
answer = model.invoke(prompt)
print(answer)


 The context provided does not mention the purchase of sunburn gel.


In [31]:
dataloader.get(['Data\\monopoly.pdf16'])

{'ids': ['Data\\monopoly.pdf16'],
 'embeddings': None,
 'documents': ['you can move anywhere you want on the board! \nIf you get sent to jail during your move (either by landing on the "Go \nto Jail" space or by rolling doubles three times in a row) then your turn'],
 'uris': None,
 'data': None,
 'metadatas': [{'id': 'Data\\monopoly.pdf16',
   'page': 1,
   'page_label': '2',
   'source': 'Data\\monopoly.pdf'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}